Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [6]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)
df = pd.concat([pd.DataFrame.from_records(ham), pd.DataFrame.from_records(spam)], ignore_index=True)


skipped 2248.2004-09-23.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [5]:
import re

def preprocessor(e):
    e = re.sub(r'[^a-zA-Z]', ' ', e)
    e = e.lower()
    return e

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.

vectorizer = CountVectorizer(preprocessor=preprocessor)

# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.

x_train, x_test, y_train, y_test = train_test_split(df['content'], df['category'], test_size=0.2)


# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.

x_train = vectorizer.fit_transform(x_train)


# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.

model = LogisticRegression()


# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.

x_test = vectorizer.transform(x_test)
model.fit(x_train, y_train)
predictions = model.predict(x_test)


# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.

score = accuracy_score(y_test, predictions)
matrix = confusion_matrix(y_test, predictions)
report = classification_report(y_test, predictions)

print(score)
print(matrix)
print(report)



0.9806576402321083
[[719  14]
 [  6 295]]
              precision    recall  f1-score   support

         ham       0.99      0.98      0.99       733
        spam       0.95      0.98      0.97       301

    accuracy                           0.98      1034
   macro avg       0.97      0.98      0.98      1034
weighted avg       0.98      0.98      0.98      1034



Step 4.

In [9]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.

features = vectorizer.get_feature_names_out()
print(features)


# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.

importance = model.coef_[0]
print(importance)


# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.

positive = []
negative = []
for i, word in enumerate(features):
    if importance[i] > 0:
        positive.append((word, importance[i]))
    else:
        negative.append((word, importance[i]))

positive.sort(key=lambda x: x[1], reverse=True)
negative.sort(key=lambda x: x[1])

print(positive[:10])
print(negative[:10])



['aa' 'aaa' 'aaas' ... 'zzezrjok' 'zzo' 'zzso']
[-2.48234327e-01  5.58267552e-04  9.55179731e-05 ...  1.25831209e-02
  1.57344710e-02  8.46892402e-02]
[('http', np.float64(0.9679909384138936)), ('no', np.float64(0.8151926754405777)), ('prices', np.float64(0.7518705882306813)), ('hello', np.float64(0.7475599514655454)), ('paliourg', np.float64(0.7326519415701492)), ('removed', np.float64(0.7318915713060604)), ('money', np.float64(0.7153618540615267)), ('remove', np.float64(0.670162754016483)), ('off', np.float64(0.6550971643239295)), ('more', np.float64(0.630668048187054))]
[('attached', np.float64(-1.528419294198215)), ('enron', np.float64(-1.5279424258940655)), ('daren', np.float64(-1.5138486623621297)), ('thanks', np.float64(-1.3703339680047333)), ('deal', np.float64(-1.238856209441285)), ('pictures', np.float64(-1.2138685437089014)), ('meter', np.float64(-1.1665361224378825)), ('xls', np.float64(-1.1463704825303327)), ('doc', np.float64(-1.0879952003070021)), ('hpl', np.float64(-1.0

Submission
1. Upload the jupyter notebook to Forage.

All Done!